In [55]:
import pandas as pd
import numpy as np

from pandas import DataFrame, Series

# 時系列データ
「Pythonによるデータ分析入門 - Numpy,pandasを使ったデータ処理」の10章の時系列データをもとにまとめました。

ある時点において観測されたデータは、どのようなものでも時系列を構成している。多くの時系列は**一定頻度**である。しかし、扱う時系列は固定の単位時間やオフセットがないような**不規則**なものであっても構わない。

# 目次

 - [10.1 日付、時間のデータ型とツール](#10.1-日付、時間のデータ型とツール)
     - [10.1.1 文字列とdatetimeの変換](#10.1.1-文字列とdatetimeの変換)
 - [10.2 時系列の基本](#10.2-時系列の基本)
     - [10.2.1 インデックス参照、データの選択、サブセットの抽出](#10.2.1-インデックス参照、データの選択、サブセットの抽出)
     - [10.2.2 重複したインデックスをもつ時系列](#10.2.2-重複したインデックスをもつ時系列)
 - [10.3 日付範囲、頻度、シフト](#10.3-日付範囲、頻度、シフト)
     - [10.3.1 日付範囲の生成](#10.3.1-日付範囲の生成:-pandas.date_range(start,-end,-periods,-freq))
     - [10.3.2 頻度と日付オフセット](#10.3.2-頻度と日付オフセット)
     - [10.3.3 データの前方と後方へのシフト](#10.3.3-データの前方と後方へのシフト:-時系列データ.shift(periods,-freq,-axis))
 - (省略)10.4 タイムゾーンを扱う
     - 10.4.1 ローカライゼーションと変換
     - 10.4.2 タイムゾーンを意識したタイムスタンプオブジェクト
     - 10.4.3 別のタイムゾーンとの演算

---
# 10.1 日付、時間のデータ型とツール

**datetimeモジュールに含まれるデータ型**

| データ型 | 説明 |
|:----------:|:-----:|
| `date` | グレゴリオ暦の日付(年、月、日)の情報を持つ。 |
| `time` | 1日の時間の情報(時、分、秒、マイクロ秒)を持つ。 |
| `datetime` | 日付と時間の両方の情報を持つ。 |
| `timedelta` | ２つの`datetime`型の値の差を、日、秒、マイクロ秒で表す。 |

In [3]:
from datetime import datetime

now = datetime.now()
now

datetime.datetime(2017, 7, 15, 19, 35, 12, 34124)

`datetime.datetime(年, 月, 日, 時, 分, 秒, マイクロ秒)`

In [4]:
now.year, now.month, now.day

(2017, 7, 15)

### datetimeオブジェクト間の差の計算
`datetime.timedelta`は、`datetime`オブジェクト間の差を表します。

In [10]:
delta = datetime(2017, 7, 15) - datetime(1995, 6, 21, 19, 35)
delta

datetime.timedelta(8059, 15900)

`datetime.timedelta(何日, 何秒)`

In [12]:
delta.days, delta.seconds

(8059, 15900)

### timedeltaの計算

In [14]:
from datetime import timedelta

start = datetime(2011, 1, 7)
start + timedelta(12)

datetime.datetime(2011, 1, 19, 0, 0)

In [16]:
start - 2 * timedelta(12)

datetime.datetime(2010, 12, 14, 0, 0)

## 10.1.1 文字列とdatetimeの変換

<div style="text-align:center">
### `datetime`オブジェクト/pandasのタイムスタンプ→文字列
</div>
`datetime`オブジェクトとpandasのタイムスタンプ(`Timesstamp`)オブジェクトは、`str`や`strftime`メソッドを使って書式を指定することで、文字列で表現することができる。
### `datetimeオブジェクト.strftime(書式)`

In [17]:
stamp = datetime(2011, 1, 3)

In [18]:
str(stamp)

'2011-01-03 00:00:00'

In [22]:
stamp.strftime('%Y-%m-%d')

'2011-01-03'

<div style="text-align:center">
### 文字列→`datetime`オブジェクト/pandasのタイムスタンプ
</div>
### `datetime.strptime（文字列, 書式)`

In [20]:
value = "2011-01-03"

In [21]:
datetime.strptime(value, '%Y-%m-%d')

datetime.datetime(2011, 1, 3, 0, 0)

In [23]:
datestrs = ['7/6/2011', '8/6/2011']

In [24]:
[datetime.strptime(x, '%m/%d/%Y') for x in datestrs]

[datetime.datetime(2011, 7, 6, 0, 0), datetime.datetime(2011, 8, 6, 0, 0)]

### parse(文字列)
`datetime.strptime`は、特定の書式で日付をパースするのに最も適した方法である。しかし、一般的な書式を使う場合は、毎回各時間の書式を指定するのは面倒。

In [25]:
from dateutil.parser import parse

parse('2011-01-03')

datetime.datetime(2011, 1, 3, 0, 0)

In [26]:
parse('Jan 31, 1997 10:45 PM')

datetime.datetime(1997, 1, 31, 22, 45)

In [27]:
# 月よりも日が前に現れたときのパースは、dayfirst=Trueを指定
parse('6/21/2011', dayfirst=True)

datetime.datetime(2011, 6, 21, 0, 0)

### pd.to_datetime(文字列orリストorタップルor1-Nの配列orシリーズ)
 - arg: integer or float or string or datetime or リスト or Tuple or 1-Nの配列 or Series

pandasのデータフレームのインデックスや列にある日付の配列を扱えるように作れた。
[pd.to_datetimeの詳細](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html)

In [28]:
datestrs = ['7/6/2011', '8/6/2011']

In [29]:
pd.to_datetime(datestrs)

DatetimeIndex(['2011-07-06', '2011-08-06'], dtype='datetime64[ns]', freq=None)

In [31]:
# 欠損値(Noneや空文字など)を扱う
idx = pd.to_datetime(datestrs + [None])
idx

DatetimeIndex(['2011-07-06', '2011-08-06', 'NaT'], dtype='datetime64[ns]', freq=None)

In [33]:
pd.isnull(idx)

array([False, False,  True], dtype=bool)

日付と時間の書式

| 書式 | 説明 |
|:-----:|:-----:|
| `%Y` | 4桁の年 |
| `%y` | ２桁の年 |
| `%m` | ２桁の月[01, 12] |
| `%d` | ２桁の日[01,31] |
| `%H` | 時間(24時間)[00, 23] |
| `%I` | 時間(12時間)[01, 12] |
| `%M` | ２桁の分[00, 59] |
| `%S` | 秒[00, 61] (60と61はうるう秒を考慮したもの) |
| `%w` | 曜日を表す整数[0(日曜), 6(土曜)] |

---
# 10.2 時系列の基本
pandasにおける最も基本的な時系列オブジェクトは、タイムスタンプによってインデックス付けされたシリーズである。このタイムスタンプはPythonの文字列やdatetimeオブジェクトを使う。

In [37]:
from datetime import datetime

dates = [
    datetime(2011, 1, 2), datetime(2011, 1, 5), datetime(2011, 1, 7), 
    datetime(2011, 1, 8), datetime(2011, 1, 10), datetime(2011, 1, 12)
]
ts = Series(np.random.randn(6), index=dates)
ts

2011-01-02   -0.878902
2011-01-05   -0.078415
2011-01-07   -0.427183
2011-01-08    1.100024
2011-01-10    1.809965
2011-01-12    0.277677
dtype: float64

内部では、`datetime`オブジェクトが`DatetimeIndex`クラスに渡され、変数`ts`はこの時点で時系列(`TimeSeries`)型になる。

In [38]:
type(ts)

pandas.core.series.Series

In [39]:
ts.index

DatetimeIndex(['2011-01-02', '2011-01-05', '2011-01-07', '2011-01-08',
               '2011-01-10', '2011-01-12'],
              dtype='datetime64[ns]', freq=None)

In [48]:
stamp = ts.index[0]
stamp

Timestamp('2011-01-02 00:00:00')

## 10.2.1 インデックス参照、データの選択、サブセットの抽出
時系列は、シリーズのサブクラスなので、インデックス参照・データの選択などはシリーズと同様な振る舞いをする。

### データの選択

In [49]:
stamp = ts.index[2]
ts[stamp]

-0.42718306181225002

In [50]:
ts['1/10/2011']

1.8099645005818084

In [51]:
ts['20110110']

1.8099645005818084

### サブセットの抽出
長い時系列の場合、ある年やある年月を指定して、データの一部分を選択できる

In [52]:
longer_ts = Series(np.random.randn(1000), index=pd.date_range('1/1/2000', periods=1000))
longer_ts

2000-01-01   -0.933219
2000-01-02   -0.456159
2000-01-03    1.213587
2000-01-04   -2.249837
2000-01-05   -0.121399
2000-01-06    0.120252
2000-01-07    1.667014
2000-01-08   -0.452382
2000-01-09    0.123710
2000-01-10   -0.581417
2000-01-11   -0.398408
2000-01-12    1.923971
2000-01-13   -1.429598
2000-01-14    0.043977
2000-01-15    0.968895
2000-01-16   -0.218424
2000-01-17    1.914892
2000-01-18   -1.008917
2000-01-19   -0.670680
2000-01-20    0.564774
2000-01-21   -0.663499
2000-01-22   -1.543444
2000-01-23    0.456847
2000-01-24    1.258326
2000-01-25    0.020148
2000-01-26    0.828681
2000-01-27   -0.698024
2000-01-28    0.182161
2000-01-29   -0.470119
2000-01-30    0.300146
                ...   
2002-08-28    1.644348
2002-08-29    0.736103
2002-08-30   -0.215498
2002-08-31    0.084696
2002-09-01   -0.238179
2002-09-02   -0.117243
2002-09-03    0.767720
2002-09-04    1.351074
2002-09-05    2.215219
2002-09-06    0.152384
2002-09-07    1.056759
2002-09-08   -0.640112
2002-09-09 

In [53]:
# 2001年のデータ
longer_ts['2001']

2001-01-01    0.394088
2001-01-02    0.594825
2001-01-03   -1.683998
2001-01-04   -0.754066
2001-01-05    0.150672
2001-01-06    0.326947
2001-01-07    0.304799
2001-01-08   -0.072869
2001-01-09    0.461722
2001-01-10    1.120106
2001-01-11   -1.177085
2001-01-12   -1.304187
2001-01-13   -0.146277
2001-01-14   -1.346021
2001-01-15   -0.212737
2001-01-16    1.466339
2001-01-17   -0.727520
2001-01-18   -1.448845
2001-01-19   -1.729323
2001-01-20    0.008132
2001-01-21   -0.227207
2001-01-22   -0.614349
2001-01-23    0.125862
2001-01-24    1.191822
2001-01-25   -2.463811
2001-01-26   -0.584736
2001-01-27    0.649463
2001-01-28    0.844499
2001-01-29   -0.308816
2001-01-30    0.250401
                ...   
2001-12-02    1.128321
2001-12-03    0.254759
2001-12-04   -0.874159
2001-12-05   -0.840031
2001-12-06    0.087336
2001-12-07   -0.251760
2001-12-08   -0.932517
2001-12-09   -0.980080
2001-12-10    0.091575
2001-12-11   -0.718474
2001-12-12   -1.071208
2001-12-13    1.102614
2001-12-14 

In [54]:
# 2001年５月のデータ
longer_ts['2001-05']

2001-05-01    0.445308
2001-05-02   -0.814985
2001-05-03    1.858886
2001-05-04   -1.975515
2001-05-05   -1.858425
2001-05-06    0.061223
2001-05-07    1.041752
2001-05-08    0.628756
2001-05-09    0.447677
2001-05-10    0.604242
2001-05-11    1.129539
2001-05-12    0.174651
2001-05-13    0.795280
2001-05-14    0.171184
2001-05-15   -0.617713
2001-05-16   -2.440714
2001-05-17   -0.721279
2001-05-18    1.759132
2001-05-19    1.110302
2001-05-20   -1.589561
2001-05-21    1.077661
2001-05-22   -0.412318
2001-05-23   -0.135070
2001-05-24   -1.169620
2001-05-25   -0.336480
2001-05-26   -1.456525
2001-05-27    0.184122
2001-05-28   -0.809803
2001-05-29   -0.519860
2001-05-30    0.885821
2001-05-31   -0.564621
Freq: D, dtype: float64

### DataFrame版

In [56]:
dates = pd.date_range('1/1/2000', periods=100, freq='W-WED')
long_df = DataFrame(np.random.randn(100, 4), index=dates, columns=['Colorado', 'Texas', 'New York', 'Ohio'])

In [57]:
long_df.ix['5-2001']

,Colorado,Texas,New York,Ohio
2001-05-02,-0.668551,0.420849,-0.190000,-1.777934
2001-05-09,-1.521044,1.216055,-1.408504,-0.372913
2001-05-16,1.239138,0.606125,-1.140517,0.460187
2001-05-23,-0.247856,0.908995,0.450816,-0.440556
2001-05-30,-2.234902,-0.027185,-0.425483,0.988520


## 10.2.2 重複したインデックスを持つ時系列

In [61]:
dates = pd.DatetimeIndex(['1/1/2000', '1/2/2000', '1/2/2000', '1/2/2000', '1/3/2000'])
dup_ts = Series(np.arange(5), index=dates)
dup_ts

2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int64

In [64]:
# インデックスが一意でないことを確認
dup_ts.index.is_unique

False

In [68]:
# タイムスタンプが重複していない場合→スカラー値
dup_ts['1/3/2000']

4

In [67]:
# タイムスタンプが重複している場合→データの集合
dup_ts['1/2/2000']

2000-01-02    1
2000-01-02    2
2000-01-02    3
dtype: int64

### 一意でないタイムスタンプを持つデータを集約

In [70]:
# level=0 (インデックス参照している階層を指定している。この場合はインデックス)
grouped = dup_ts.groupby(level=0)

In [71]:
grouped.mean()

2000-01-01    0
2000-01-02    2
2000-01-03    4
dtype: int64

In [72]:
grouped.count()

2000-01-01    1
2000-01-02    3
2000-01-03    1
dtype: int64

---
# 10.3 日付範囲、頻度、シフト
pandasの時系列では基本的に不規則なデータを想定している。つまり、一定頻度であることを想定していない。しかし、たとえ時系列に欠損値を追加することになったとしても、日次、月次、15分おきなどの一定頻度のデータを扱える方が望ましい場合がある。幸いpandasは、標準的な時系列の頻度を扱う昨日や、再サンプリングなどのツールを一式持っている。

In [79]:
ts

2011-01-02   -0.878902
2011-01-05   -0.078415
2011-01-07   -0.427183
2011-01-08    1.100024
2011-01-10    1.809965
2011-01-12    0.277677
dtype: float64

In [80]:
# resampleメソッドを呼ぶことで一定頻度の時系列データに変換することができる
# ※再サンプリングは後ほど独立した節で説明する
ts.resample('D')

DatetimeIndexResampler [freq=<Day>, axis=0, closed=left, label=left, convention=start, base=0]

## 10.3.1 日付範囲の生成: `pandas.date_range(start, end, periods, freq)`
`pandas.date_range(start, end, periods, freq)`は一定の頻度に従う指定した長さの`DatetimeIndex`を生成する

 - `start`: 開始日
 - `end`: 終了日
 - `periods`: 生成する日数
 - `freq`: 頻度

In [82]:
index = pd.date_range('4/1/2012', '6/1/2012')
index

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20',
               '2012-04-21', '2012-04-22', '2012-04-23', '2012-04-24',
               '2012-04-25', '2012-04-26', '2012-04-27', '2012-04-28',
               '2012-04-29', '2012-04-30', '2012-05-01', '2012-05-02',
               '2012-05-03', '2012-05-04', '2012-05-05', '2012-05-06',
               '2012-05-07', '2012-05-08', '2012-05-09', '2012-05-10',
               '2012-05-11', '2012-05-12', '2012-05-13', '2012-05-14',
               '2012-05-15', '2012-05-16', '2012-05-17', '2012-05-18',
               '2012-05-19', '2012-05-20', '2012-05-21', '2012-05-22',
               '2012-05-23', '2012-05-24', '2012-05-25', '2012-05-26',
      

In [83]:
pd.date_range(start='4/1/2012', periods=20)

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20'],
              dtype='datetime64[ns]', freq='D')

開始日と終了日は、生成された日付のインデックスにおいて厳密な境界になる。たとえば、各月の最終営業日をインデックスに含ませたい場合は、'BM'(business end of month)を頻度として指定するが、このときは、指定した開始日と終了日に一致するか、範囲の内側に含まれる日付だけが結果に含まれる。

In [84]:
pd.date_range('1/1/2000', '12/1/2000', freq='BM')

DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-28',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-29', '2000-10-31', '2000-11-30'],
              dtype='datetime64[ns]', freq='BM')

In [85]:
# pd.date_range()は、デフォルトでは開始と終了のタイムスタンプを(もしあれば)保存する
pd.date_range('5/2/2012 12:56:32', periods=5)

DatetimeIndex(['2012-05-02 12:56:32', '2012-05-03 12:56:32',
               '2012-05-04 12:56:32', '2012-05-05 12:56:32',
               '2012-05-06 12:56:32'],
              dtype='datetime64[ns]', freq='D')

In [86]:
# タイムスタンプを午前零時に標準化したい場合は、normalize=True
pd.date_range('5/2/2012 12:56:32', periods=5, normalize=True)

DatetimeIndex(['2012-05-02', '2012-05-03', '2012-05-04', '2012-05-05',
               '2012-05-06'],
              dtype='datetime64[ns]', freq='D')

## 10.3.2 頻度と日付オフセット
pandasにおいて、頻度は**基準頻度**と乗数の組み合わせで構成されている。普通は、毎月の場合を'M'、毎時の場合を'H'のように文字列で基準頻度を参照する。基準頻度には、**日付オフセット**と一般的に呼ばれるオブジェクトが定義されている。

In [87]:
from pandas.tseries.offsets import Hour, Minute

# 毎時の頻度
hour = Hour()
hour

<Hour>

In [88]:
# 整数を指定することで日付オフセットの倍数を定義
four_hours = Hour(4)
four_hours

<4 * Hours>

In [89]:
pd.date_range('1/1/2000', '1/3/2000 23:59', freq='4h')

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 04:00:00',
               '2000-01-01 08:00:00', '2000-01-01 12:00:00',
               '2000-01-01 16:00:00', '2000-01-01 20:00:00',
               '2000-01-02 00:00:00', '2000-01-02 04:00:00',
               '2000-01-02 08:00:00', '2000-01-02 12:00:00',
               '2000-01-02 16:00:00', '2000-01-02 20:00:00',
               '2000-01-03 00:00:00', '2000-01-03 04:00:00',
               '2000-01-03 08:00:00', '2000-01-03 12:00:00',
               '2000-01-03 16:00:00', '2000-01-03 20:00:00'],
              dtype='datetime64[ns]', freq='4H')

In [90]:
Hour(2) + Minute(30)

<150 * Minutes>

In [95]:
print(pd.date_range('1/1/2000', periods=10, freq='1h30min'))
print(pd.date_range('1/1/2000', periods=10, freq=Hour(1) + Minute(30)))

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 01:30:00',
               '2000-01-01 03:00:00', '2000-01-01 04:30:00',
               '2000-01-01 06:00:00', '2000-01-01 07:30:00',
               '2000-01-01 09:00:00', '2000-01-01 10:30:00',
               '2000-01-01 12:00:00', '2000-01-01 13:30:00'],
              dtype='datetime64[ns]', freq='90T')
DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 01:30:00',
               '2000-01-01 03:00:00', '2000-01-01 04:30:00',
               '2000-01-01 06:00:00', '2000-01-01 07:30:00',
               '2000-01-01 09:00:00', '2000-01-01 10:30:00',
               '2000-01-01 12:00:00', '2000-01-01 13:30:00'],
              dtype='datetime64[ns]', freq='90T')


時系列の基準頻度

| 文字 | オフセットクラス | 説明 |
|:-----:|:------------------|:-------|
| D | `Day` | 暦通りの日次 |
| B | `BusinessDay` | 毎営業日 |
| H | `Hour` | 毎時 |
| Tまたはmin | `Minute` | 毎分 |
| S | `Second` | 毎秒 |
| Lまたはms | `Milli` | 毎ミリ秒(1秒の1/1000) |
| U | Micro | 毎マイクロ秒(１秒の1/1000000) |
| M | `MonthEnd` | 暦通りの月末 |
| BM | `BusinessMonthEnd` | 月の最終営業日 |
| MS | `MonthBegin` | 暦通りの月初ごと |
| BMS | `BusinessMonthBegin` | 月の営業開始日ごと |
| W-MON, W-TUE, ... | `Week` | 毎週指定した曜日ごと: MON(月)、TUE(火)、...　 |
| WOM-1MON, WOM-2MON, ... | `WeekOfMonth` | 月の第１〜４週日の指定した曜日ごと。例えば、WOM-3FRIの場合は、毎月第3金曜日。 |
| Q-JAN, Q-FEB, ... | `QuarterEnd` | 指定した月に年度が終わる前提で、四半期の暦通りの月末ごと: JAN(1月)、FEB(２月)、MAR(３月)、APR(４月)、... |
| BQ-JAN, BQ-FEB, ... | `BusinessQuarterEnd` | 指定した月に年度が終わる前提で、四半期の最終営業日ごと |
| QS-JAN, QS-FEB, ... | `QuarterBegin` | 指定した月に年度が終わる前提で、四半期の暦通りの月初ごと |
| BQS-JAN, BQS-FEB, ... | `BusinessQuarterEnd` | 指定した月に年度が終わる前提で、四半期の最終営業日ごと |

## 10.3.3 データの前方と後方へのシフト: `時系列データ.shift(periods, freq, axis)`
[pandas.DataFrame.shift](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shift.html)  
ここでいう「シフト」は、データを時間的に前方や後方へ移動させることを指す。シリーズでもデータフレームも`shift`メソッドを持っていて、単純な前方、後方へのシフトを行う。このときインデックスは変わらない。

In [108]:
ts = Series(np.random.randn(4), index=pd.date_range('1/1/2000', periods=4, freq='M'))
ts

2000-01-31   -0.643082
2000-02-29   -0.457722
2000-03-31   -0.657434
2000-04-30   -0.440094
Freq: M, dtype: float64

In [100]:
ts.shift(1)

2000-01-31         NaN
2000-02-29    0.470174
2000-03-31    0.663022
2000-04-30   -0.598547
Freq: M, dtype: float64

### 時系列のパーセント変化を計算
$$
変化率 = \frac {Y_{t+1} - Y_{t}}{Y_{t}}
$$

In [104]:
# ΔY/Y
ts/ts.shift(1) -1

2000-01-31         NaN
2000-02-29   -0.290862
2000-03-31   -2.107719
2000-04-30   -1.419115
Freq: M, dtype: float64

### タイムスタンプ(インデックス)もシフトさせる: 時系列データ.shift(periods, freq)

In [106]:
# 三ヶ月シフトする
ts.shift(3, freq='M')

2000-04-30    0.470174
2000-05-31    0.663022
2000-06-30   -0.598547
2000-07-31    1.428122
Freq: M, dtype: float64

In [112]:
# ３日シフト
ts.shift(3, freq='D')

2000-02-03   -0.643082
2000-03-03   -0.457722
2000-04-03   -0.657434
2000-05-03   -0.440094
dtype: float64

### 10.3.3.1 オフセットを使用して日付をシフトする
pandasの日付オフセットは`datetime`やタイムスタンプオブジェクトに対しても使うことができる。

In [110]:
from pandas.tseries.offsets import Day, MonthEnd

now = datetime(2011, 11, 17)

In [115]:
now + 3 * Day()

Timestamp('2011-11-20 00:00:00')

In [116]:
now + MonthEnd()

Timestamp('2011-11-30 00:00:00')

In [117]:
now + MonthEnd(2)

Timestamp('2011-12-31 00:00:00')

#### `オフセットオブジェクト.rollforward()`と`オフセットオブジェクト.rollback()`

In [120]:
offset = MonthEnd()
offset.rollforward(now)

Timestamp('2011-11-30 00:00:00')

In [121]:
offset.rollback(now)

Timestamp('2011-10-31 00:00:00')

In [122]:
# groupbyメソッドと一緒に使う
ts = Series(np.random.randn(20), index=pd.date_range('1/15/2000', periods=20, freq='4d'))
ts

2000-01-15   -0.329445
2000-01-19   -1.172113
2000-01-23    0.549152
2000-01-27    0.380388
2000-01-31   -1.355119
2000-02-04   -1.195955
2000-02-08   -0.424088
2000-02-12   -0.576696
2000-02-16    1.001201
2000-02-20    0.035975
2000-02-24   -0.010602
2000-02-28    1.180081
2000-03-03    1.525230
2000-03-07   -1.307874
2000-03-11   -0.859251
2000-03-15   -0.186853
2000-03-19    0.420273
2000-03-23   -1.332621
2000-03-27    0.319369
2000-03-31    1.268241
Freq: 4D, dtype: float64

In [126]:
# 以下の処理は、resampleメソッドを使うと簡単で早く同じことができる。
ts.groupby(offset.rollforward).mean()

2000-01-31   -0.385427
2000-02-29    0.001416
2000-03-31   -0.019186
dtype: float64